## Working with rHEALpix in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/04_rhealpix.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

For working with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/), [vgridpandas document](https://vgridpandas.gishub.vn)

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/) if needed.

In [ ]:
# %pip install vgrid --upgrade

### latlon2rhealpix

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2rhealpix

lat = 10.775276
lon = 106.706797
res = 12
rhealpix_id = latlon2rhealpix(lat, lon, res)
rhealpix_id

### rHEALPix ID to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.rhealpix2geo import rhealpix2geojson

rhealpix_geojson = rhealpix2geojson(rhealpix_id)
rhealpix_geojson

### Vector to rHEALPix

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.vector2dggs.vector2rhealpix import vector2rhealpix

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
rhealpix_geojson = vector2rhealpix(file_path, resolution=10, compact=False, predicate = "intersects")
# Change compact to True to see the compact version of the rHEALPix cells

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

rhealpix_layer = folium.GeoJson(
rhealpix_geojson,
style_function=lambda x: {
    "fillColor": "blue",
    "color": "black",
    "fillOpacity": 0.3,
    "weight": 1,
},
tooltip=folium.GeoJsonTooltip(
    fields=["rhealpix", "resolution", "cell_area"],
    aliases=["rHEALPix ID", "Resolution", "Area (m²)"],
    style="""
        background-color: white;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
),
).add_to(m)

m.fit_bounds(rhealpix_layer.get_bounds())

# Display the map
m

### rHEALPix Compact

In [ ]:
from vgrid.conversion.dggscompact import rhealpixcompact

rhealpix_compacted = rhealpixcompact(
    rhealpix_geojson, rhealpix_id="rhealpix"
)

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

rhealpix_compacted_layer = folium.GeoJson(
    rhealpix_compacted,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["rhealpix", "resolution", "cell_area"],
        aliases=["rHEALPix ID", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(rhealpix_compacted_layer.get_bounds())

# Display the map
m

### rHEALPix Expand

In [ ]:
from vgrid.conversion.dggscompact import rhealpixexpand

rhealpix_expanded = rhealpixexpand(
    rhealpix_geojson, resolution=11
)

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

rhealpix_expanded_layer = folium.GeoJson(
    rhealpix_expanded,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["rhealpix", "resolution", "cell_area"],
        aliases=["rHEALPix ID", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(rhealpix_expanded_layer.get_bounds())

# Display the map
m

### rHEALPix Binning

In [ ]:
from vgrid.binning.rhealpixbin import rhealpixbin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
rhealpix_bin = rhealpixbin(file_path, resolution=10, stats="count")
rhealpix_bin 